In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install flair

     |████████████████████████████████| 337kB 2.8MB/s 
     |████████████████████████████████| 19.7MB 49.4MB/s 
     |████████████████████████████████| 798kB 37.0MB/s 
     |████████████████████████████████| 1.3MB 37.9MB/s 
     |████████████████████████████████| 983kB 37.9MB/s 
     |████████████████████████████████| 1.1MB 37.2MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 2.9MB 35.5MB/s 
     |████████████████████████████████| 890kB 34.9MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116677 sha256=216a2bfd2999019cc8f60ab73502213e1e01558fc0d7ce91a46677de647b7179
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993195 sha256=ffade0bf58b6783180a25bb99b2d6b64b7c7f0a30c646c0781d57a5bfcb77380
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e

In [3]:
#important libraries
import pandas as pd
import numpy as np
import nltk
import re
#importing stopwords is optional, in this case it decreased accuracy
#from nltk.corpus import stopwords
import itertools
import time

In [4]:
# Load data
import pandas as pd

#col_names = ['sentiment','id','date','query_string','user','text']
data_path = '/content/drive/My Drive/Twitter data/text_emotion.csv'


tweet_data = pd.read_csv(data_path, encoding="ISO-8859-1").sample(frac=1) # .sample(frac=1) shuffles the data
tweet_data = tweet_data[['sentiment', 'content']] # Disregard other columns
tweet_data.head()

,sentiment,content
7635,worry,@ajlnike09 omg wtf why is there a porn site fo...
21844,neutral,Surf's up this week - starting this afternoon
7542,sadness,We're not gonna be able to talk again
26372,happiness,this is my second to last biology and the firs...
15029,sadness,@theAV8TR dont quit.


In [5]:
df_train = pd.read_csv("/content/drive/My Drive/Twitter data/archive/train.txt", delimiter=';', header=None, names=['sentence','label'])
df_test = pd.read_csv("/content/drive/My Drive/Twitter data/archive/test.txt", delimiter=';', header=None, names=['sentence','label'])
df_val = pd.read_csv("/content/drive/My Drive/Twitter data/archive/val.txt", delimiter=';', header=None, names=['sentence','label'])

In [6]:
df = pd.concat([df_train,df_test,df_val])
df['label'].unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [8]:
df.head(10)

,sentence,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,fear
8,i have been with petronas for years i feel tha...,joy
9,i feel romantic too,love


In [9]:
#change column label to sentiment
df = df.rename(columns = {"label":"sentiment"})

In [10]:
#check distribution of data based on labels
print("Distribution of data based on labels: ",df.sentiment.value_counts())

Distribution of data based on labels:  joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: sentiment, dtype: int64


In [ ]:
#tweet_data['sentiment'].unique()

In [ ]:
#change column content to sentence
#tweet_data = tweet_data.rename(columns = {"content":"sentence"})

In [ ]:
#check distribution of data based on labels
#print("Distribution of data based on labels: ",tweet_data.sentiment.value_counts())

In [ ]:
# merge both df and tweet_data
# final_df = pd.concat([df, tweet_data])
# len(final_df)


In [ ]:
#final_df.sentiment.value_counts()

In [ ]:
#remove certain categories from the data: fun, relief, 
#combine both hate and anger
# combine happiness with enthusiasm and joy
# combine empty with sadness

#final_df['sentiment'] = final_df['sentiment'].replace({"joy":"happiness", "hate":"anger", "enthusiasm":"happiness", "hate":"anger", "empty":"sadness", "boredom":"sadness"})

In [ ]:
#final_df.sentiment.value_counts()

In [11]:
import re

allowed_chars = ' AaBbCcDdEeFfGgHhIiJjKkLlMmNnOoPpQqRrSsTtUuVvWwXxYyZz0123456789~`!@#$%^&*()-=_+[]{}|;:",./<>?'
punct = '!?,.@#'
maxlen = 280

def preprocess(text):
    return ''.join([' ' + char + ' ' if char in punct else char for char in [char for char in re.sub(r'http\S+', 'http', text, flags=re.MULTILINE) if char in allowed_chars]])[:maxlen]

In [12]:
df['sentence'] = df['sentence'].apply(preprocess)

In [ ]:
#final_df.to_csv(data_dir + '/final.csv', header= True, index = False)

In [ ]:
#data_dir

In [13]:
# change the categories of sentiment into numerical form
df["sentiment"] = df["sentiment"].astype('category')
df["sentiment"] = df["sentiment"].cat.codes

In [14]:
df['sentiment'] = '__label__' + df['sentiment'].astype(str)

In [15]:
df.head(10)

,sentence,sentiment
0,i didnt feel humiliated,__label__4
1,i can go from feeling so hopeless to so damned...,__label__4
2,im grabbing a minute to post i feel greedy wrong,__label__0
3,i am ever feeling nostalgic about the fireplac...,__label__3
4,i am feeling grouchy,__label__0
5,ive been feeling a little burdened lately wasn...,__label__4
6,ive been taking or milligrams or times recomme...,__label__5
7,i feel as confused about life as a teenager or...,__label__1
8,i have been with petronas for years i feel tha...,__label__2
9,i feel romantic too,__label__3


In [17]:
cols = ["sentiment", "sentence"]
df = df[cols]

In [18]:
import os

# Create directory for saving data if it does not already exist
data_dir = '/content/drive/My Drive/Twitter data/emotions/new_data/'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

# Save a percentage of the data (you could also only load a fraction of the data instead)

df.iloc[0:int(len(df)*0.8)].to_csv(data_dir + '/train.csv', index=False, header=False)
df.iloc[int(len(df)*0.8):int(len(df)*0.9)].to_csv(data_dir + '/test.csv', index=False, header=False)
df.iloc[int(len(df)*0.9):int(len(df)*1.0)].to_csv(data_dir + '/dev.csv', index=False, header=False)

In [19]:
from flair.data import Corpus
from flair.datasets import ClassificationCorpus

# this is the folder in which train, test and dev files reside
data_folder = data_dir

# load corpus containing training, test and dev data
corpus: Corpus = ClassificationCorpus(data_folder,
                                      test_file='test.csv',
                                      dev_file='dev.csv',
                                      train_file='train.csv')

2020-11-09 06:01:22,378 Reading data from /content/drive/My Drive/Twitter data/emotions/new_data
2020-11-09 06:01:22,379 Train: /content/drive/My Drive/Twitter data/emotions/new_data/train.csv
2020-11-09 06:01:22,385 Dev: /content/drive/My Drive/Twitter data/emotions/new_data/dev.csv
2020-11-09 06:01:22,387 Test: /content/drive/My Drive/Twitter data/emotions/new_data/test.csv


In [20]:
#from flair.data_fetcher import NLPTaskDataFettcher
#from pathlib import Path

#corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_dir), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')

In [21]:
label_dict = corpus.make_label_dictionary()

2020-11-09 06:01:22,636 Computing label dictionary. Progress:


100%|██████████| 18000/18000 [00:15<00:00, 1146.79it/s]

2020-11-09 06:01:38,500 [b'4,i', b'0,im', b'3,i', b'0,i', b'4,ive', b'5,ive', b'1,i', b'2,i', b'5,i', b'2,ive', b'2,on', b'4,im', b'4,when', b'2,im', b'2,finding', b'5,im', b'1,im', b'1,when', b'0,id', b'4,id', b'1,fear', b'2,ill', b'2,id', b'3,im', b'1,ive', b'3,is', b'4,the', b'0,when', b'0,ive', b'2,when', b'0,my', b'1,ill', b'0,this', b'2,my', b'1,as', b'4,my', b'1,waiting', b'3,id', b'0,a', b'2,imdoing', b'1,while', b'4,a', b'3,ive', b'0,we', b'2,is', b'4,ill', b'0,sometime', b'0,ill', b'2,there', b'0,in', b'1,stranded', b'4,after', b'1,earth', b'4,for', b'0,the', b'2,during', b'0,at', b'2,arriving', b'1,staying', b'2,the', b'0,getting', b'1,during', b'2,a', b'1,watched', b'4,death', b'4,during', b'2,having', b'1,tutorial', b'0,heated', b'1,in', b'1,once', b'2,went', b'0,first', b'1,going', b'3,ill', b'1,every', b'1,my', b'0,during', b'0,watching', b'4,this', b'2,boy', b'0,no', b'1,occured', b'1,id', b'0,having', b'0,being', b'2,iv', b'0,realizing', b'1,the', b'5,ill', b'1,on', b'

In [22]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings

word_embeddings = [WordEmbeddings('glove'),
                    #FlairEmbeddings('news-forward'),
                    #FlairEmbeddings('news-backward')
                  ]

2020-11-09 06:01:39,130 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpwmpk1xe8


100%|██████████| 160000128/160000128 [00:07<00:00, 22456670.81B/s]

2020-11-09 06:01:46,606 copying /tmp/tmpwmpk1xe8 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2020-11-09 06:01:46,930 removing temp file /tmp/tmpwmpk1xe8
2020-11-09 06:01:47,318 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpij_4vtlo


100%|██████████| 21494764/21494764 [00:01<00:00, 12172654.77B/s]

2020-11-09 06:01:49,436 copying /tmp/tmpij_4vtlo to cache at /root/.flair/embeddings/glove.gensim
2020-11-09 06:01:49,467 removing temp file /tmp/tmpij_4vtlo


In [23]:
from flair.embeddings import DocumentRNNEmbeddings

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

In [24]:
from flair.models import TextClassifier

classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

In [25]:
from flair.trainers import ModelTrainer

trainer = ModelTrainer(classifier, corpus)

In [ ]:
trainer.train('model-saves2',
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=8,
              max_epochs=20)

2020-11-06 11:58:01,246 ----------------------------------------------------------------------------------------------------
2020-11-06 11:58:01,249 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
    )
    (word_reprojection_map): Linear(in_features=100, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=115, bias=True)
  (loss_function): CrossEntropyLoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-11-06 11:58:01,250 ----------------------------------------------------------------------------------------------------
2020-11-06 11:58:01,251 Corpus: "Corpus: 16000 train + 2000 dev + 2000 test sentences"
2020-11-06 11:58:01,253 ----------------------------------------------------------------------------------------------------
2020-11-06 11

{'dev_loss_history': [1.945324420928955,
  1.9326533079147339,
  1.559191107749939,
  1.3066920042037964,
  1.1271493434906006,
  1.0238040685653687,
  0.9342482089996338,
  0.8552535176277161,
  0.8065276145935059,
  0.739918053150177,
  0.703685462474823,
  0.6572102904319763,
  0.625537097454071,
  0.6310291886329651,
  0.614797055721283,
  0.5761505365371704,
  0.549270749092102,
  0.5605600476264954,
  0.5371593832969666,
  0.5069844126701355],
 'dev_score_history': [0.3843,
  0.3552,
  0.521,
  0.6278,
  0.6784,
  0.6954,
  0.7255,
  0.7495,
  0.754,
  0.7776,
  0.7876,
  0.8001,
  0.7981,
  0.7976,
  0.8086,
  0.8136,
  0.8317,
  0.8246,
  0.8372,
  0.8387],
 'test_score': 0.8475,
 'train_loss_history': [2.158373929977417,
  2.0070091712474825,
  1.8302668118476868,
  1.6537962676286697,
  1.521107925415039,
  1.4191316524744033,
  1.3354870924949647,
  1.2652165032625198,
  1.195945608139038,
  1.1580571936368942,
  1.1074739416837693,
  1.0796274485588073,
  1.0727766813635826

In [29]:
from flair.data import Sentence

classifier = TextClassifier.load('/content/drive/My Drive/Twitter data/model-saves2/emotion-model.pt')

pos_sentence = Sentence(preprocess('I love Python!'))
neg_sentence = Sentence(preprocess('Python is the worst!'))

classifier.predict(pos_sentence)
classifier.predict(neg_sentence)

print(pos_sentence.labels, neg_sentence.labels)

2020-11-09 06:02:52,135 loading file /content/drive/My Drive/Twitter data/model-saves2/emotion-model.pt
[2,i (0.2218)] [0,when (0.0835)]


In [37]:
# show predictions

label_dict = {'0': 'Anger', '1': 'Fear', '2': 'Joy', '3': 'Love', '4': 'Sadness', '5': 'Surprise'}

sentence = Sentence(preprocess('I love Python!'))
classifier.predict(sentence)

if len(sentence.labels) > 0:
  print('your sentence is predicted to portray ' + label_dict[sentence.labels[0].value[0]] + ' with', sentence.labels[0].score*100, '% confidence') 

your sentence is predicted to portray Joy with 22.17828929424286 % confidence
